### サンプルデータ
https://github.com/thinkitcojp/TensorFlowDL-samples/blob/master/save_model/mnist.py



In [2]:
#-*- coding:utf-8 -*-
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

#mnistデータを格納したオブジェクトを呼び出す
mnist = input_data.read_data_sets("data/", one_hot=True)

#入力データを定義
x = tf.placeholder(tf.float32, [None, 784])
#入力層から中間層
w_1 = tf.Variable(tf.truncated_normal([784, 64], stddev=0.1), name="w1")
b_1 = tf.Variable(tf.zeros([64]), name="b1")
h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)

#中間層から出力層
w_2 = tf.Variable(tf.truncated_normal([64, 10], stddev=0.1), name="w2")
b_2 = tf.Variable(tf.zeros([10]), name="b2")
out = tf.nn.softmax(tf.matmul(h_1, w_2) + b_2)

#誤差関数
y = tf.placeholder(tf.float32, [None, 10])
loss = tf.reduce_mean(tf.square(y - out))

#訓練
global_step = tf.Variable(0, name='global_step', trainable=False)
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss, global_step=global_step)

#評価
correct = tf.equal(tf.argmax(out,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

#初期化
init =tf.global_variables_initializer()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


- ここまでは多層パーセプトロンと同じ

In [3]:
saver = tf.train.Saver(max_to_keep = 3)

- TensorFlowでもっとも基本的なモデルの保存方法として`tf.train.Saver()`がある
    - TensorFlowによって作成されたgraphとVariableの値を保存、読み込むためのクラス
- max_to_keepはモデルの最新x件を保存しておくかのオプション

In [4]:
with tf.Session() as sess:
   

    ckpt_state = tf.train.get_checkpoint_state('ckpt/')
    if ckpt_state:
        last_model = ckpt_state.model_checkpoint_path
        saver.restore(sess,last_model)
        print("model was loaded:", last_model)
    else:
        sess.run(init)
        print("initialized.")

    #テストデータをロード    
    test_images = mnist.test.images    
    test_labels = mnist.test.labels

    
    last_step = sess.run(global_step)
    for i in range(1000):
        step = last_step + i      
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x:train_images ,y:train_labels})

        
        if (step+1) % 100 == 0:
            acc_val = sess.run(accuracy ,feed_dict={x:test_images, y:test_labels})
            print('Step %d: accuracy = %.2f' % (step + 1, acc_val))
            saver.save(sess, 'ckpt/my_model', global_step = step + 1, write_meta_graph=False)


initialized.
Step 100: accuracy = 0.42
Step 200: accuracy = 0.60
Step 300: accuracy = 0.76
Step 400: accuracy = 0.83
Step 500: accuracy = 0.86
Step 600: accuracy = 0.87
Step 700: accuracy = 0.88
Step 800: accuracy = 0.89
Step 900: accuracy = 0.89
Step 1000: accuracy = 0.90


- write_meta_graphは書き込み時にグラフのメタ情報を一緒に書くかどうかのフラグ
    - 今回のケースではソースコードがあるため必要ない

In [5]:
!ls -la ckpt/

total 1232
drwxr-xr-x   9 amoro  staff     306  5 15 20:45 .
drwxr-xr-x  10 amoro  staff     340  5 15 21:20 ..
-rw-r--r--   1 amoro  staff     169  5 15 20:45 checkpoint
-rw-r--r--   1 amoro  staff  203564  5 15 20:45 my_model-1000.data-00000-of-00001
-rw-r--r--   1 amoro  staff     232  5 15 20:45 my_model-1000.index
-rw-r--r--   1 amoro  staff  203564  5 15 20:45 my_model-800.data-00000-of-00001
-rw-r--r--   1 amoro  staff     232  5 15 20:45 my_model-800.index
-rw-r--r--   1 amoro  staff  203564  5 15 20:45 my_model-900.data-00000-of-00001
-rw-r--r--   1 amoro  staff     232  5 15 20:45 my_model-900.index
